In [1]:
import pandas as pd
import numpy as np
from Bio import SearchIO

In [2]:
taxa_file_path = '/home/gridsan/cdoering/LaubLab_shared/assembly_summary.txt'
FT_file_path = '/home/gridsan/cdoering/LaubLab_shared/All_RefSeq_Assemblies_Accessions.parquet'

In [3]:
#Assembly taxid and species taxid information for every genbank and refseq genome and store in dictionary
assemblySums = pd.read_csv(taxa_file_path,sep = '\t',skiprows = 1,usecols = ['#assembly_accession','species_taxid'])
Acc2Taxid = assemblySums.set_index('#assembly_accession')['species_taxid'].to_dict()  

In [4]:
#Read in combined feature table
FT = pd.read_parquet(FT_file_path,columns=['non-redundant_refseq','assembly','genomic_accession'])

In [5]:
phmmer_file = '/home/gridsan/cdoering/TaxFinder/phmmer_result/CmdTAC_phmmer.txt'
#Read in phmmer hits as dictionary
phmmer_hits = {}
for record in SearchIO.parse(phmmer_file,'hmmer3-tab'):
    hits = {x.id for x in record.hits}
    phmmer_hits[record.id] = hits
all_hits = {hit for one_set_of_hits in phmmer_hits.values() for hit in one_set_of_hits}

In [23]:
FT.reset_index(inplace=True,drop=True)

In [24]:
#Filter FT to only include hits
FT_hits = FT[FT['non-redundant_refseq'].isin(all_hits)]

In [9]:
FT_hits

,non-redundant_refseq,assembly,genomic_accession
1633,WP_153829390.1,GCF_009659645.1,NZ_RJJK01000019.1
10073,WP_258992913.1,GCF_024787905.1,NZ_JANUII010000001.1
341,WP_134850079.1,GCF_018205535.1,NZ_JADRHD010000002.1
1893,WP_128334817.1,GCF_030344115.1,NZ_JATAVY010000015.1
7183,WP_122432578.1,GCF_900586945.1,NZ_UTZI01000040.1
...,...,...,...
6287,WP_199950141.1,GCF_016508605.1,NZ_JADVOO010000010.1
3769,WP_299502381.1,GCF_028287845.1,NZ_JAQBOW010000163.1
1817,WP_232112265.1,GCF_022382355.1,NZ_JAAGDM010000001.1
5655,WP_070579015.1,GCF_018475425.1,NZ_JADIAJ010000014.1


In [ ]:
prot_names = ['PD-T4-A','PD-T4-C']
buff = 1

In [ ]:
#Search for systems among hits. If multi-protein system, proteins must all be within buffer distance of each other
found_systems = []
for genomic_acc, group in FT_hits.groupby('genomic_accession'):
    assembly_acc = group.reset_index().at[0,'assembly']
    skip_search = False
    indices_in_ft = {}
    for protein_id, hits_set in phmmer_hits.items():
        overlapping_hits = set(group['non-redundant_refseq']).intersection(hits_set)
        if len(overlapping_hits) == 0: #If any of the proteins are not present stop this specific search
            skip_search = True
            break
        indices_in_ft[protein_id] = group[group['non-redundant_refseq'].isin(overlapping_hits)].index
    # if len(indices_in_ft) > 0:
    #     print(indices_in_ft)
    if skip_search:
        continue
        
    if (len(indices_in_ft) == 1) and (len(prot_names) == 1):
        found_systems.append((assembly_acc,Acc2Taxid[assembly_acc]))
        continue
                            
    search_dist = len(prot_names) + buff
    near_another = {protein_id: False for protein_id in indices_in_ft}
    for protein_i, i_indices in indices_in_ft.items():
        i_array = np.array(i_indices)
        for protein_j, j_indices in indices_in_ft.items():
            if protein_i != protein_j:
                j_array = np.array(j_indices)
                dist_matrix = np.abs(i_array[:,None] - j_array)
                if (dist_matrix <= search_dist).any():
                    near_another[protein_i] = True
                    near_another[protein_j] = True

    if all(near_another.values()):
        found_systems.append((assembly_acc,Acc2Taxid[assembly_acc]))

In [ ]:
found_systems

[('GCF_019670525.1', 38313),
 ('GCF_000489035.1', 670),
 ('GCF_000500525.1', 670),
 ('GCF_005391925.1', 562),
 ('GCF_005394765.1', 562),
 ('GCF_003569565.2', 280145),
 ('GCF_005398985.1', 562),
 ('GCF_004311065.1', 244366),
 ('GCF_004311085.1', 244366),
 ('GCF_004312845.1', 573),
 ('GCF_004313225.1', 573),
 ('GCF_008990765.1', 280145),
 ('GCF_008992365.1', 280145),
 ('GCF_008993755.1', 280145),
 ('GCF_009001395.1', 40215),
 ('GCF_009013415.1', 108980),
 ('GCF_009013715.1', 106654),
 ('GCF_014645715.1', 1259555),
 ('GCF_021601705.1', 1785145),
 ('GCF_023708065.1', 2942288),
 ('GCF_026009115.1', 573),
 ('GCF_026010935.1', 573),
 ('GCF_900774705.1', 1463165),
 ('GCF_905198805.1', 237779),
 ('GCF_905331655.1', 670),
 ('GCF_929618295.1', 548),
 ('GCF_948193485.1', 670),
 ('GCF_001050875.1', 669),
 ('GCF_000737515.1', 1356852),
 ('GCF_001700835.1', 670),
 ('GCF_001559215.2', 548),
 ('GCF_002022045.1', 1117645),
 ('GCF_002386305.1', 582),
 ('GCF_002796525.1', 548),
 ('GCF_003294855.2', 648),
